# Resilience & Performance

Distributed systems fail. Design for failure with:
- timeouts
- retries with backoff
- circuit breakers
- bulkheads

Performance focuses on the critical path: reduce latency, avoid N+1 queries, and use caching wisely.
